In [1]:
import mne
import numpy as np
from tensorflow import keras

2023-09-03 20:07:05.689892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 20:07:06.987846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
def get_data(picks,epochs):
    data = epochs.get_data(picks=picks)
    # data = data * 1e5
    scaler = mne.decoding.Scaler(scalings='mean')
    data = scaler.fit_transform(data)
    data = np.moveaxis(data,1,2)
    return data

In [ ]:
def preprocess():
    labels_list=[]
    left_fist_both_fist = [3,4,7,8,11,12]
    right_fist_both_feet = [5,6,9,10,13,14]
    for i in range (1, 110):
        if i != 100 and i != 92 and i != 88 :
            for j in range(3,15):
                raw=mne.io.read_raw_edf(f"eeg-motor-movementimagery-dataset-1.0.0/S{str(i).rjust(3, '0')}/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}.edf", preload=True, verbose=False)
                events=mne.events_from_annotations(raw,verbose=False)
                # print(raw.ch_names)
                raw.set_eeg_reference()
                event_id=[1, 2, 3]
                epochs = mne.Epochs(raw, events[0], event_id, tmin = 0.0, tmax = 4.0 - 1.0 / raw.info['sfreq'], baseline = (0.0, 0.5), on_missing ='warn',verbose=False)
                data,labels=[],[]
                data.append(get_data(['Fc1.', 'Fc2.'],epochs))
                data.append(get_data(['Fc3.', 'Fc4.'],epochs))
                data.append(get_data(['Fc5.', 'Fc6.'],epochs))
                data.append(get_data(['C1..', 'C2..'],epochs))
                data.append(get_data(['C3..', 'C4..'],epochs))
                data.append(get_data(['C5..', 'C6..'],epochs))
                data.append(get_data(['Cp1.', 'Cp2.'],epochs))
                data.append(get_data(['Cp3.', 'Cp4.'],epochs))
                data.append(get_data(['Cp5.', 'Cp6.'],epochs))
                events=epochs.events[:,-1]
                data = np.concatenate(data)
                for l in range(0,9):
                    labels.append(events)
                labels = np.concatenate(labels)
                if j in left_fist_both_fist:
                    for e in range(len(labels)):
                        if labels[e] == 1:
                            labels[e] = 0
                        if labels[e] == 2:
                            labels[e] = 1
                        if labels[e] == 3:
                            labels[e] = 2
                if j in right_fist_both_feet:
                    for e in range(len(labels)):
                        if labels[e] == 1:
                            labels[e] = 0
                        if labels[e] == 2:
                            labels[e] = 3
                        if labels[e] == 3:
                            labels[e] = 4
                labels_list.append(labels)
                for r in range(len(data)):
                    np.save(f"EEG MI ROI-F/datasets/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}{str(r).rjust(3, '0')}.npy", data[r])
    labels_list = np.concatenate(labels_list)
    labels_list = keras.utils.to_categorical(labels_list, 5)
    np.save('EEG MI ROI-F/labels_list.npy', labels_list)
preprocess()
labels_list = np.load('EEG MI ROI-F/labels_list.npy')
print(labels_list)
print(len(labels_list))

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Using data from preloaded Raw for 30 events and 640 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
Using data from preloaded Raw for 30 events and 640 original time points ...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Using data from preloaded Raw for 30 events and 640 original tim

In [3]:
data_test = np.load('EEG MI ROI-F/datasets/S109R14000.npy')
print(data_test)
print(data_test.shape)

[[-0.12143097  0.03991732]
 [-0.12963756  0.08171737]
 [-0.14270111  0.00658309]
 ...
 [ 0.22240844  0.34733292]
 [ 0.18288282  0.12087186]
 [ 0.21905881 -0.00188274]]
(640, 2)


In [2]:
labels_list = np.load('EEG MI ROI-F/labels_list.npy')
print(labels_list)
print(len(labels_list))

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
343323
